In [1]:
## Reads in a folder of data and creates a frequency table for primer pairs that match the reads
## Basic, only for separate reads fastq documents

from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
import sys
import os
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Seq import Seq
import subprocess
import sys
import csv
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

SeqFilePath = '/Users/annechen/Downloads/CRISPR Pools Data/test2'
PrimerID = '/Users/annechen/Downloads/sgRNAs for multiplexed CRISPR - Primer Sequences Only.csv'

output_folder = join(SeqFilePath, 'testout')
## Attempting to put data in separate output_folder
if not os.path.isdir(output_folder): 
    os.mkdir(output_folder)

def primerfreq(SeqFile, start_index):
    
    ############################################################################
    ## Load CSV File of Primer IDs into pandas dataframe
    
    primer = pd.read_csv(PrimerID)
    primer['Frequency'] = 0
    
    ############################################################################
    ## Make a dictionary for sequences
    ## Read in the fastq file  

    SeqDict = {}
    freq = {}

    with open(SeqFile, 'rU') as handle:
        for record in SeqIO.parse(handle, "fastq"):
            SeqDict[str(record.id)] = str(record.seq)
            #ProbeseqList.append(str(record.id)) ## Also make a list of all of the probe sequences for writing the output file.

    ############################################################################
    ##  Make Dictionary by looping through list and count the frequencies

    for record in SeqDict:
        seq = str(SeqDict[record])
        added = False
        newcode = ''
        if seq[-8:] != 'GGGGGGGG':
            for i in freq:
                if i[0:15] == seq[0:15]:
                    freq[i] = freq[i] + 1
                    added = True
                else:
                    newcode = seq[0:15]
            if added == False:
                freq[newcode] = 1

    ############################################################################
    ##  Convert previous dictionary to dataframe 

    fTable = pd.DataFrame.from_dict(freq, orient='index', columns=['Frequency'])
    fTable = fTable.sort_values('Frequency',ascending=False)
    fTable['ID'] = '-'
    
    ############################################################################
    ##  Enter matching primer ID if exists on fTable
    ##  Enter frequency of appearance on primer dataframe

    for seq_index, seq_row in fTable.iterrows():
        for prim_index, prim_row in primer.iterrows():
            primerID = primer.iloc[prim_index,1]
            #print seq_index + ' ' + primerID[start_index+0:start_index+15]
            if seq_index == primerID[start_index:start_index+15]:
                fTable.at[seq_index,'ID'] = primer.iloc[prim_index,0]
                primer.iloc[prim_index,2] = fTable.at[seq_index,'Frequency']
    ##pd.set_option('display.max_rows', 208) ##allows you to view entirety of fTable w/o truncation

    ##print (fTable)
    
    primer = primer.sort_values('Frequency',ascending=False)
    primer = primer.reset_index()
    del primer['index']
    
    ############################################################################
    ##  Export primer frequency data as csv, managing file path
    
    csv_file = join (output_folder, os.path.splitext(os.path.basename(SeqFile))[0] + '_primerMatch.csv')
    
    ##csv_file = join(output_folder, os.path.splitext(SeqFile)[0] + '_primerMatch.csv')

    primer.to_csv(csv_file, sep='\t', index=False)
    primercsv = pd.read_csv('primerMatch.csv',sep='\t')
    #print(primercsv)


for f in listdir(SeqFilePath):
    if not f.startswith('.'):
        ## In R2 file, sequencing left off first base, readjusted index to shift one right
        if '_R2_' in f:
            ##primerfreq(f,1)
            primerfreq(join(SeqFilePath,f),1)
        if '_R1_' in f:
            ##primerfreq(f,0)

            primerfreq(join(SeqFilePath, f),0)

In [15]:
## Code for csv of paired primers from a list of seperate reads

sep_reads = '/Users/annechen/Downloads/CRISPR Pools Data/test'

############################################################################
## Finding matching files in the folder to designate R1 as forward and R2 as reverse

fastq_data = []

for f1 in listdir(sep_reads):
    if '_R1_' in f1 and not f1.startswith('.'):
        forward_f = f1
        for f2 in listdir(sep_reads):
            if '_R2_' in f2 and not f2.startswith('.'):
                if f1.replace('_R1','') == f2.replace('_R2',''):
                    reverse_f = f2
                    
                    ############################################################################
                    ## Creating dataframe with ID, forward, and reverse reads
                    
                    with open (join(sep_reads,f1), 'rU') as forward:
                        for forward_data in SeqIO.parse(forward, 'fastq'):
                            fastq_data.append([str(forward_data.id), str(forward_data.seq)])
                    
                    fastq_df = pd.DataFrame(fastq_data, columns = ['read_ID', 'forward'])
                    index_pos = 0
                    
                    fastq_df['reverse'] = ''
                    with open (join(sep_reads,f2),'rU') as reverse:
                        for reverse_data in SeqIO.parse(reverse, 'fastq'):
                            rev_code = str(reverse_data.seq)
                            ##print fastq_df.at[index_pos, 'read_ID']
                            if str(reverse_data.id) == fastq_df.at[index_pos,'read_ID']:
                                fastq_df.at[index_pos, 'reverse'] = rev_code
                                index_pos += 1
                                
                    ############################################################################            
                    ## Making file name            
                    initial_file_name = os.path.basename(f1.replace('_R1',''))
                    combined_file = join(sep_reads, os.path.splitext(initial_file_name)[0] + '_combined.csv')
                    
                    ##print (initial_file_name)
                    ##print os.path.basename(initial_file_name)
                    ##os.path.splitext(os.path.basename(SeqFile))[0] 
                    ## Making csv file
                    fastq_df.to_csv(combined_file, sep='\t', index=False)
                    ##fastqcsv = pd.read_csv()
                    

1-1_S2_L001_001.fastq
1-1_S2_L001_001.fastq
36_S31_L001_001.fastq
36_S31_L001_001.fastq


In [ ]:
## Working doc that reads in a folder of combined reads and outputs data (only works on one doc at a time)

PrimerID = '/Users/annechen/Downloads/sgRNAs for multiplexed CRISPR - Primer Sequences Only.csv'
combined = '/Users/annechen/Downloads/CRISPR Pools Data/test2/results/1-1_S2_L001_001_combined.csv'
path = '/Users/annechen/Downloads/CRISPR Pools Data/test2/results'


comb_reads = pd.read_csv(combined, sep='\t')
primer = pd.read_csv(PrimerID)
primer.head()

############################################################################
## Create 6 dataframes to hold different information

concordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
discordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
onematch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
nomatch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
freqTable = pd.DataFrame(columns = ['Forward Prim ID', 'Reverse Prim ID', 'Frequency'])
aggData = {}

############################################################################
## Load PrimerID values into freqTable

## Need to change this part of code according to name of primer sequences
for index, row in primer.iterrows():
    if index == 30:
        break
    freqTable = freqTable.append({'Forward Prim ID':primer.iloc[index,0],'Reverse Prim ID':primer.iloc[index+30, 0], 'Frequency':0}, ignore_index=True)
                         
############################################################################
## Iterate through combined list to sort read pairs

for seq_index, seq_row in comb_reads.iterrows():
    
    fseq = comb_reads.iloc[seq_index,1]
    rseq = comb_reads.iloc[seq_index,2]
    
    ##Gfor and Grev = True if there is G-Tail at end of sequence, False if there is no G-tail
    Gfor = fseq[-8:] == 'GGGGGGGG'
    Grev = rseq[-8:] == 'GGGGGGGG'
    ##fID and rID will be changed later if a match is found and there is no G-tail
    fID = 'no forward match'
    rID = 'no reverse match'
    
    ############################################################################
    ## Match each read without a G-tail to a read ID
    
    if not Gfor or not Grev:
        for p_index, p_row in primer.iterrows():
            primerID = primer.iloc[p_index,1]

            if not Gfor and fseq[0:15] == primerID[0:15]:
                fID = primer.iloc[p_index, 0]
            if not Grev and rseq[0:15] == primerID[1:16]:
                rIDcode = primer.iloc[p_index, 0]
                ## Need to change this part of code according to name of primer sequences (manually matches rev code to forward by subtracting 30)
                rID = rIDcode[0:2] + str(int(rIDcode[2:])-30)
    
    
    ############################################################################
    ## Sort read into one of four categories depending on presence of G-Tail and PrimerID Match
    
    if Gfor and Grev:
        nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])
        
    elif fID == rID:
        ## print 'concordant ' + comb_reads.iloc[seq_index, 0][-4:]
        concordant = pd.concat([concordant, comb_reads.iloc[[seq_index]]])
        
        ## Input info into concordant pair frequency table
        for index, row in freqTable.iterrows():
            if fID == freqTable.iloc[index,0]:
                freqTable.iloc[index,2] += 1
        
    elif fID != 'no forward match' and rID != 'no reverse match':
        ## print 'discordant ' + comb_reads.iloc[seq_index, 0][-4:]
        discordant = pd.concat([discordant, comb_reads.iloc[[seq_index]]])
        
    elif fID != 'no forward match' or rID != 'no reverse match':
        onematch = pd.concat([onematch, comb_reads.iloc[[seq_index]]])
        ## print 'one match ' + comb_reads.iloc[seq_index, 0][-4:]
        
    else:
        nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])

## Sort freqTable for primer freq from largest to smallest
freqTable = freqTable.sort_values('Frequency', ascending=False)
freqTable = freqTable.reset_index()
del freqTable['index']

############################################################################
## Creating aggregate data dictionary and load into new dataframe

aggData['Concordant'] = len(concordant.index)
aggData['Discordant'] = len(discordant.index)
aggData['One ID Match'] = len(onematch.index)
aggData['No ID Match'] = len(nomatch.index)

aggData_df = pd.DataFrame(aggData.items(), columns=['Pair Type', 'Frequency'])

############################################################################
## Create csv files for each dataframe

concord_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_concordant_file.csv')
concordant.to_csv(concord_csv, sep='\t', index=False)

discord_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_discordant_file.csv')
discordant.to_csv(discord_csv, sep='\t', index=False)

onematch_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_single_match_file.csv')
onematch.to_csv(onematch_csv, sep='\t', index=False)

nomatch_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_no_matches_file.csv')
nomatch.to_csv(nomatch_csv, sep='\t', index=False)

freqTable_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_concordant_freqTable.csv')
freqTable.to_csv(freqTable_csv, sep='\t', index=False)

aggData_df_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_aggregate_data.csv')
aggData_df.to_csv(aggData_df_csv, sep='\t', index=False)


In [181]:
## Reads in a folder of seperate fastq reads
## Creates output of combined reads from fastq files and 6 docs of data:
## 6 docs: list of concordant, discordant, single match, and no match reads, a frequency table of concordant reads, and an aggregate table of the number of reads of each type

from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
import sys
import os
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Seq import Seq
import subprocess
import sys
import csv
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

sep_reads = '/Users/annechen/Downloads/CRISPR Pools Data/test'
PrimerID = '/Users/annechen/Downloads/sgRNAs for multiplexed CRISPR - Primer Sequences Only.csv'

############################################################################
## Creating output folder to put results in

output_folder = join(sep_reads, 'results')
## Put data in separate output_folder
if not os.path.isdir(output_folder): 
    os.mkdir(output_folder)
    
############################################################################
## Create function to analyze combined reads csv file and split data into the 6 docs
                    
def pairAnalyze(CombSeqFile, PrimerIDFile):
    
    comb_reads = pd.read_csv(CombSeqFile, sep='\t')
    primer = pd.read_csv(PrimerIDFile)
    
    ############################################################################
    ## Create 6 dataframes to hold different information

    concordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    discordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    onematch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    nomatch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    freqTable = pd.DataFrame(columns = ['Forward Prim ID', 'Reverse Prim ID', 'Frequency'])
    aggData = {}
    
    ############################################################################
    ## Load PrimerID values into freqTable

    ## Need to change this part of code according to name of primer sequences
    for index, row in primer.iterrows():
        if index == 30:
            break
        freqTable = freqTable.append({'Forward Prim ID':primer.iloc[index,0],'Reverse Prim ID':primer.iloc[index+30, 0], 'Frequency':0}, ignore_index=True)

    ############################################################################
    ## Iterate through combined list to sort read pairs

    for seq_index, seq_row in comb_reads.iterrows():

        fseq = comb_reads.iloc[seq_index,1]
        rseq = comb_reads.iloc[seq_index,2]

        ##Gfor and Grev = True if there is G-Tail at end of sequence, False if there is no G-tail
        Gfor = fseq[-8:] == 'GGGGGGGG'
        Grev = rseq[-8:] == 'GGGGGGGG'
        ##fID and rID will be changed later if a match is found and there is no G-tail
        fID = 'no forward match'
        rID = 'no reverse match'
        
        ############################################################################
        ## Match each read without a G-tail to a read ID

        if not Gfor or not Grev:
            for p_index, p_row in primer.iterrows():
                primerID = primer.iloc[p_index,1]

                if not Gfor and fseq[0:15] == primerID[0:15]:
                    fID = primer.iloc[p_index, 0]
                if not Grev and rseq[0:15] == primerID[1:16]:
                    rIDcode = primer.iloc[p_index, 0]
                    ## Need to change this part of code according to name of primer sequences (manually matches rev code to forward by subtracting 30)
                    rID = rIDcode[0:2] + str(int(rIDcode[2:])-30)

        ############################################################################
        ## Sort read into one of four categories depending on presence of G-Tail and PrimerID Match

        if Gfor and Grev:
            nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])

        elif fID == rID:
            ## print 'concordant ' + comb_reads.iloc[seq_index, 0][-4:]
            concordant = pd.concat([concordant, comb_reads.iloc[[seq_index]]])

            ## Input info into concordant pair frequency table
            for index, row in freqTable.iterrows():
                if fID == freqTable.iloc[index,0]:
                    freqTable.iloc[index,2] += 1
                
        elif fID != 'no forward match' and rID != 'no reverse match':
            ## print 'discordant ' + comb_reads.iloc[seq_index, 0][-4:]
            discordant = pd.concat([discordant, comb_reads.iloc[[seq_index]]])

        elif fID != 'no forward match' or rID != 'no reverse match':
            onematch = pd.concat([onematch, comb_reads.iloc[[seq_index]]])
            ## print 'one match ' + comb_reads.iloc[seq_index, 0][-4:]

        else:
            nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])

    ## Sort freqTable for primer freq from largest to smallest
    freqTable = freqTable.sort_values('Frequency', ascending=False)
    freqTable = freqTable.reset_index()
    del freqTable['index']

    ############################################################################
    ## Creating aggregate data dictionary and load into new dataframe

    aggData['Concordant'] = len(concordant.index)
    aggData['Discordant'] = len(discordant.index)
    aggData['One ID Match'] = len(onematch.index)
    aggData['No ID Match'] = len(nomatch.index)

    aggData_df = pd.DataFrame(aggData.items(), columns=['Pair Type', 'Frequency'])

    ############################################################################
    ## Create csv files for each dataframe

    concord_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_concordant_file.csv')
    concordant.to_csv(concord_csv, sep='\t', index=False)

    discord_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_discordant_file.csv')
    discordant.to_csv(discord_csv, sep='\t', index=False)

    onematch_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_single_match_file.csv')
    onematch.to_csv(onematch_csv, sep='\t', index=False)

    nomatch_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_no_matches_file.csv')
    nomatch.to_csv(nomatch_csv, sep='\t', index=False)

    freqTable_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_concordant_freqTable.csv')
    freqTable.to_csv(freqTable_csv, sep='\t', index=False)

    aggData_df_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_aggregate_data.csv')
    aggData_df.to_csv(aggData_df_csv, sep='\t', index=False)

############################################################################
## Fill results folder with combined reads pooling paired separate reads together

for f1 in listdir(sep_reads):
    if '_R1_' in f1 and not f1.startswith('.'):
        forward_f = f1
        for f2 in listdir(sep_reads):
            if '_R2_' in f2 and not f2.startswith('.'):
                if f1.replace('_R1','') == f2.replace('_R2',''):
                    
                    fastq_data = []
                    reverse_f = f2
                    
                    ############################################################################
                    ## Creating dataframe with ID, forward, and reverse reads
                    
                    with open (join(sep_reads,f1), 'rU') as forward:
                        for forward_data in SeqIO.parse(forward, 'fastq'):
                            fastq_data.append([str(forward_data.id), str(forward_data.seq)])
                    
                    fastq_df = pd.DataFrame(fastq_data, columns = ['read_ID', 'forward'])
                    index_pos = 0
                    
                    fastq_df['reverse'] = ''
                    with open (join(sep_reads,f2),'rU') as reverse:
                        for reverse_data in SeqIO.parse(reverse, 'fastq'):
                            rev_code = str(reverse_data.seq)
                            ##print fastq_df.at[index_pos, 'read_ID']
                            if str(reverse_data.id) == fastq_df.at[index_pos,'read_ID']:
                                fastq_df.at[index_pos, 'reverse'] = rev_code
                                index_pos += 1
                                
                    ############################################################################            
                    ## Making file name            
                    initial_file_name = os.path.basename(f1.replace('_R1',''))
                    combined_file = join(output_folder, os.path.splitext(initial_file_name)[0] + '_combined.csv')
                    
                    ## Making csv file
                    fastq_df.to_csv(combined_file, sep='\t', index=False)

## Load CSV File of Primer IDs into pandas dataframe
##primer = pd.read_csv(PrimerID)

for f in listdir(output_folder):
    if not f.startswith('.'):
        pairAnalyze(join(output_folder, f), PrimerID)
    
    
    